## Preprocessing trajectory data

Data preprocessing is a set of activities performed to prepare data for future analysis and data mining activities.

## Load data from file

The dataset used in this tutorial is GeoLife GPS Trajectories. Available in https://www.microsoft.com/en-us/download/details.aspx?id=52367

In [1]:
import pandas as pd
import numpy as np
from pymove import MoveDataFrame

In [2]:
df = pd.read_csv('examples/geolife_sample.csv', parse_dates=['datetime'])
df.head()

,lat,lon,datetime,id
0,39.984094,116.319236,2008-10-23 05:53:05,1
1,39.984198,116.319322,2008-10-23 05:53:06,1
2,39.984224,116.319402,2008-10-23 05:53:11,1
3,39.984211,116.319389,2008-10-23 05:53:16,1
4,39.984217,116.319422,2008-10-23 05:53:21,1


In [3]:
df_move = MoveDataFrame(df, latitude="lat", longitude="lon", datetime="datetime")

In [4]:
df_move.show_trajectories_info()


======================= INFORMATION ABOUT DATASET =======================

Number of Points: 217653

Number of IDs objects: 2

Start Date:2008-10-23 05:53:05     End Date:2009-03-19 05:46:37

Bounding Box:(22.147577, 113.54884299999999, 41.132062, 121.156224)





## Filtering

The filters module provides functions to perform different types of data filtering.

Importing the module:

In [ ]:
from pymove import filters

A bounding box (usually shortened to bbox) is an area defined by two longitudes and two latitudes. The function by_bbox, filters points of the trajectories according to a especified bounding box.

In [ ]:
bbox = (22.147577, 113.54884299999999, 41.132062, 121.156224)
filt_df = filters.by_bbox(df_move, bbox)
filt_df.head()

by_datetime function filters point trajectories according to the time specified by the parameters: start_datetime and end_datetime.

In [ ]:
filters.by_datetime(df_move,start_datetime = "2009-03-19 05:45:37", end_datetime = "2009-03-19 05:46:17")

by label function filters trajectories points according to specified value and column label, set by value and label_name respectively.

In [ ]:
filters.by_label(df_move, value = 116.327219, label_name = "lon").head()

by_id function filters trajectories points according to especified trajectory id.

In [ ]:
filters.by_id(df_move, id_=5).head()

A tid is the result of concatenation between the id and date of a trajectory.
The by_tid function filters trajectory points according to the tid specified by the tid_ parameter.

In [ ]:
df_move.generate_tid_based_on_id_datatime()
filters.by_tid(df_move, "12008102305").head()

outliers function filters trajectories points that are outliers.

In [ ]:
outliers_points = filters.outliers(df_move)
outliers_points.head()

clen_duplicates function removes the duplicate rows of the Dataframe, optionally only certaind columns can be consider.

In [ ]:
filters.clean_duplicates(df_move)

clean_consecutive_duplicates function removes consecutives duplicate rows of the Dataframe. Optionally only certaind columns can be consider, this is defined by the parameter subset, in this example only the lat column is considered.

In [ ]:
filtered_df = filters.clean_consecutive_duplicates(df_move, subset = ["lat"])
len(filtered_df)

clean_nan_values function removes missing values from the dataframe.

In [ ]:
filters.clean_nan_values(df_move)
len(df_move)

clean_gps_jumps_by_distance function removes from the dataframe the trajectories points that are outliers.

In [ ]:
filters.clean_gps_jumps_by_distance(df_move)

clean_gps_nearby_points_by_distances function removes points from the trajectories when the distance between them and the point before is smaller than the parameter radius_area.

In [ ]:
filters.clean_gps_nearby_points_by_distances(df_move, radius_area = 10)

clean_gps_nearby_points_by_speed function removes points from the trajectories when the speed of travel between them
and the point before is smaller than the value set by the parameter speed_radius.

In [ ]:
filters.clean_gps_nearby_points_by_speed(df_move, speed_radius=40.0)

clean_gps_speed_max_radius function recursively removes trajectories points with speed higher than the value especifeid by the user.
    Given any point p of the trajectory, the point will be removed if one of the following happens:
    if the travel speed from the point before p to p is greater than the  max value of speed between adjacent
    points set by the user. Or the travel speed between point p and the next point is greater than the value set by
    the user. When the clening is done, the function will update the time and distance features in the dataframe and
    will call itself again.
    The function will finish processing when it can no longer find points disrespecting the limit of speed.

In [ ]:
filters.clean_gps_speed_max_radius(df_move)

clean_trajectories_with_few_points function removes from the given dataframe, trajectories with fewer points than was specified by the parameter min_points_per_trajectory.

In [ ]:
filters.clean_trajectories_with_few_points(df_move)

## Segmantation

The segmentation module are used to segment trajectories based on different parameters.

Importing the module:

In [ ]:
from pymove import segmentation

bbox_split function splits the bounding box in grids of the same size. The number of grids is defined by the parameter number_grids.

In [ ]:
bbox = (22.147577, 113.54884299999999, 41.132062, 121.156224)
segmentation.bbox_split(bbox, number_grids=4)

by_dist_time_speed functions segments the trajectories into clusters based on distance, time and speed. The distance, time and speed limits by the parameters by max_dist_between_adj_points, max_time_between_adj_points, max_speed_between_adj_points respectively. The column tid_part is added, it indicates the segment to which the point belongs to.

In [ ]:
segmentation.by_dist_time_speed(df_move, max_dist_between_adj_points=5000, 
                                max_time_between_adj_points=800,max_speed_between_adj_points=60.0)
df_move.head()

by_speed function segments the trajectories into clusters based on speed. The speed limit is defined by the parameter max_speed_between_adj_points. The column tid_speed is added, it indicates the segment to  which the point belongs to.

In [ ]:
segmentation.by_speed(df_move, max_speed_between_adj_points=70.0)
df_move.head()

by_time function segments the trajectories into clusters based on time. The time limit is defined by the parameter max_time_between_adj_points. The column tid_time is added, it indicates the segment to  which the point belongs to.

In [ ]:
segmentation.by_time(df_move, max_time_between_adj_points = 1000)
df_move.head()

segment_traj_by_max_dist function segments the trajectories into clusters based on distance. The distance limit is defined by the parameter max_dist_between_adj_points. The column tid_dist is added, it indicates the segment to which the point belongs to.

In [ ]:
segmentation.by_max_dist(df_move, max_dist_between_adj_points = 4000)
df_move.head()

## Stay point detection 

A stay point is location where a moving object has stayed for a while within a certain distance threshold. A stay point could stand different places such: a restaurant, a school, a work place.

Importing the module:

In [6]:
from pymove import stay_point_detection

stay_point_detection function converts the time data into a cyclical format. The columns hour_sin and hour_cos are added to the dataframe.

In [ ]:
stay_point_detection.create_update_datetime_in_format_cyclical(df_move)

In [ ]:
df_move.head()

create_or_update_move_stop_by_dist_time function creates or updates the stay points of the trajectories, based on distance and time metrics. The column segment_stop is added to the dataframe, it indicates the trajectory segment to  which the point belongs to. The column stop is also added, it indicates is the point represents a stop, a place where the object was stationary.

In [7]:
stay_point_detection.create_or_update_move_stop_by_dist_time(df_move, dist_radius=40, time_radius=1000)

Split trajectories by max distance between adjacent points: 40

Creating or updating distance features in meters...

...Sorting by id and datetime to increase performance

...Set id as index to increase attribution performance

(217653/217653) 100% in 00:00:00.075 - estimated end in 00:00:00.000
...Reset index

..Total Time: 0.08178186416625977
...setting id as index


C:\Users\Arina\Desktop\PyMove\pymove\preprocessing\segmentation.py:427: RuntimeWarning: invalid value encountered in greater
  dist = (move_data.at[idx, DIST_TO_PREV] > max_dist_between_adj_points)


(217653/217653) 100% in 00:00:00.082 - estimated end in 00:00:00.000
... Reseting index

Total Time: 0.09 seconds
------------------------------------------


Creating or updating distance, time and speed features in meters by seconds

...Sorting by segment_stop and datetime to increase performance

...Set segment_stop as index to a higher peformance

(5/217653) 0% in 00:00:00.191 - estimated end in 02:18:56.870
(43995/217653) 20% in 00:00:00.236 - estimated end in 00:00:00.932
(88581/217653) 40% in 00:00:00.287 - estimated end in 00:00:00.418
(130800/217653) 60% in 00:00:00.353 - estimated end in 00:00:00.234
(174825/217653) 80% in 00:00:00.506 - estimated end in 00:00:00.124
...Reset index...

..Total Time: 0.608
Create or update stop as True or False
...Creating stop features as True or False using 1000 to time in seconds
True     157738
False     59915
Name: stop, dtype: int64

Total Time: 1.05 seconds
-----------------------------------------------------



In [8]:
df_move.head()

,segment_stop,id,lat,lon,datetime,dist_to_prev,dist_to_next,dist_prev_to_next,time_to_prev,speed_to_prev,stop
0,1,1,39.984094,116.319236,2008-10-23 05:53:05,NaN,13.690153,NaN,NaN,NaN,False
1,1,1,39.984198,116.319322,2008-10-23 05:53:06,13.690153,7.403788,20.223428,1.0,13.690153,False
2,1,1,39.984224,116.319402,2008-10-23 05:53:11,7.403788,1.821083,5.888579,5.0,1.480758,False
3,1,1,39.984211,116.319389,2008-10-23 05:53:16,1.821083,2.889671,1.873356,5.0,0.364217,False
4,1,1,39.984217,116.319422,2008-10-23 05:53:21,2.889671,66.555997,68.727260,5.0,0.577934,False


create_update_move_and_stop_by_radius function creates or updates the stay points of the trajectories, based on distance. The column situation is also added, it indicates if the point represents a stop point or a moving point.

In [ ]:
stay_point_detection.create_update_move_and_stop_by_radius(df_move, radius=2)

In [ ]:
df_move.head()

## Compression

Importing the module:

In [10]:
from pymove import compression

The function below is used to reduce the size of the trajectory, the stop points are used to make the compression. 

In [11]:
compression.compress_segment_stop_to_point(df_move)

...setting mean to lat and lon...
...move segments will be dropped...
...get only segments stop...


(364/157738) 0% in 00:00:00.056 - estimated end in 00:00:24.579
(8437/157738) 5% in 00:00:00.215 - estimated end in 00:00:03.812
(15982/157738) 10% in 00:00:00.479 - estimated end in 00:00:04.255
(24336/157738) 15% in 00:00:00.816 - estimated end in 00:00:04.473
(31984/157738) 20% in 00:00:00.972 - estimated end in 00:00:03.824
(39733/157738) 25% in 00:00:01.154 - estimated end in 00:00:03.427
(48029/157738) 30% in 00:00:01.414 - estimated end in 00:00:03.230
(55252/157738) 35% in 00:00:01.642 - estimated end in 00:00:03.047
(71153/157738) 45% in 00:00:02.137 - estimated end in 00:00:02.601
(79310/157738) 50% in 00:00:02.324 - estimated end in 00:00:02.299
(95470/157738) 60% in 00:00:02.682 - estimated end in 00:00:01.749
(103968/157738) 65% in 00:00:02.962 - estimated end in 00:00:01.532
(110658/157738) 70% in 00:00:03.286 - estimated end in 00:00:01.398
(118491/157738) 75% in 00:00:03.799 - estimated end in 00:00:01.258
(126313/157738) 80% in 00:00:04.222 - estimated end in 00:00:01.

In [13]:
df_move

,segment_stop,id,lat,lon,datetime,dist_to_prev,dist_to_next,dist_prev_to_next,time_to_prev,speed_to_prev,stop,lat_mean,lon_mean
195,6,1,39.981364,116.326798,2008-10-23 10:36:01,NaN,12.442111,57.678462,NaN,NaN,True,39.991524,116.326407
558,6,1,40.010720,116.314060,2008-10-23 10:56:50,16.445867,76.470304,77.194096,5.0,3.289173,True,39.991524,116.326407
561,9,1,40.009262,116.312948,2008-10-23 10:56:55,NaN,35.598360,100.606304,NaN,NaN,True,40.013824,116.306535
1368,9,1,39.990973,116.326094,2008-10-24 00:04:28,16.068387,40.942759,56.717519,2.0,8.034193,True,40.013824,116.306535
1575,13,1,39.978484,116.326845,2008-10-24 01:45:41,NaN,14.366909,50.322414,NaN,NaN,True,39.980124,116.310749
1847,13,1,39.980909,116.308171,2008-10-24 02:28:19,21.870376,51.683385,53.596809,1508.0,0.014503,True,39.980124,116.310749
1914,15,1,39.982334,116.308818,2008-10-24 03:16:35,NaN,14.205236,145.788854,NaN,NaN,True,39.979741,116.313612
2646,15,1,39.982684,116.311197,2008-10-24 05:40:23,6.437367,73.921445,72.942149,5.0,1.287473,True,39.979741,116.313612
2667,19,1,39.981691,116.310004,2008-10-24 06:09:34,NaN,19.612423,51.388287,NaN,NaN,True,39.981541,116.310107
3039,19,1,39.979459,116.325806,2008-10-24 06:33:05,5.697464,78.265156,83.404371,5.0,1.139493,True,39.981541,116.310107
